In [12]:
import pandas as pd, numpy as np, random
import json
from pathlib import Path

raw = pd.read_csv("../data/processed/registry/fixtures/2025-2026/fixture_calendar.csv")
print("raw is_home counts:", raw["is_home"].value_counts(dropna=False))



raw is_home counts: is_home
0    380
1    380
Name: count, dtype: int64


In [13]:
import pandas as pd
df = pd.read_parquet("../data/decisions/candidates/A0/hold/gw6.parquet")
print(df[["team_id","lambda_goals_for","lambda_goals_against"]].drop_duplicates().sort_values("lambda_goals_for", ascending=False).head(50))


       team_id  lambda_goals_for  lambda_goals_against
0     03a0d347          3.173313              0.523628
5342  bcc4ae28          2.657872              2.089471
5741  ddbd77d6          2.102435              1.598836
760   1dd1f33c          1.598836              2.102435
3220  7fb5d771          1.275643              2.380847
4100  b11c2f05          1.015152              1.324822
360   1464d629          0.000000              0.731830
2080  259f237e          0.000000              0.404839
3638  8331d109          0.000000              1.188257


In [14]:
print("unique teams:", df['team_id'].nunique())
lam_sum = df.groupby('team_id')['lambda_goals_for'].first().sum()
print("sum λ_for across your teams:", lam_sum)
print(df[['team_id','lambda_goals_for','lambda_goals_against']].drop_duplicates('team_id').sort_values('lambda_goals_for', ascending=False).head(10))

unique teams: 9
sum λ_for across your teams: 11.823250999804827
       team_id  lambda_goals_for  lambda_goals_against
0     03a0d347          3.173313              0.523628
5342  bcc4ae28          2.657872              2.089471
5741  ddbd77d6          2.102435              1.598836
760   1dd1f33c          1.598836              2.102435
3220  7fb5d771          1.275643              2.380847
4100  b11c2f05          1.015152              1.324822
360   1464d629          0.000000              0.731830
2080  259f237e          0.000000              0.404839
3638  8331d109          0.000000              1.188257


In [15]:
print(df['pred_minutes'].describe())
print(df['p60'].describe())


count    6.181000e+03
mean     3.713360e+01
std      3.241707e+01
min      9.925252e-15
25%      1.698868e-14
50%      2.911730e+01
75%      7.399234e+01
max      8.370901e+01
Name: pred_minutes, dtype: float64
count    6181.000000
mean        0.521780
std         0.262442
min         0.055556
25%         0.282353
50%         0.660870
75%         0.761194
max         0.921978
Name: p60, dtype: float64


In [16]:
print(df.loc[df['pos']=="GK", 'pred_saves_mean'].describe())


count    7.940000e+02
mean     3.632303e-16
std      5.134540e-17
min      3.096709e-16
25%      3.096709e-16
50%      4.123912e-16
75%      4.123912e-16
max      4.123912e-16
Name: pred_saves_mean, dtype: float64
